In [ ]:
!pip install transformers datasets librosa torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install librosa torchaudio


In [ ]:
import torch
import librosa
from transformers import ClapProcessor, ClapModel

# Load CLAP processor and model
processor = ClapProcessor.from_pretrained("laion/clap-htsat-fused")
model = ClapModel.from_pretrained("laion/clap-htsat-fused")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

In [ ]:
audio_path = "/content/drive/MyDrive/Audio Files/Blue Monday (Crossnaders Remix) - Extended Mix.wav"  # Replace with your file path
audio, sr = librosa.load(audio_path, sr=48000, duration=30)  # Load 10 seconds of audio

# Preprocess audio for CLAP
audio_inputs = processor(audios=audio, return_tensors="pt")

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
# Extract audio embeddings
with torch.no_grad():
    audio_embeddings = model.get_audio_features(**audio_inputs)

In [ ]:
genres = [
    "This audio belongs to the Pop genre, characterized by catchy melodies and mainstream appeal.",
    "This audio belongs to the Rock genre, featuring electric guitars and strong rhythms.",
    "This audio belongs to the Classical genre, with orchestral arrangements and timeless compositions.",
    "This audio belongs to the Jazz genre, marked by improvisation and soulful melodies.",
    "This audio belongs to the Hip Hop genre, driven by rhythmic beats and lyrical flow.",
    "This audio belongs to the Electronic genre, defined by synthesized sounds and digital production.",
    "This audio belongs to the Experimental genre, pushing creative boundaries with unconventional sounds.",
    "This audio belongs to the Instrumental genre, focusing on melodies without vocal elements.",
    "This audio belongs to the Folk genre, rooted in traditional storytelling and acoustic sounds.",
    "This audio belongs to the Chiptune/Glitch genre, featuring retro video game-inspired sounds and glitchy effects."
]



In [ ]:
# Function to generate text embeddings for labels
def get_text_embeddings(labels):
    embeddings = {}
    for label in labels:
        inputs = processor(text=label, return_tensors="pt")
        with torch.no_grad():
            embeddings[label] = model.get_text_features(**inputs)
    return embeddings


genre_embeddings = get_text_embeddings(genres)


In [ ]:
import os
import json

# Define the directory path
output_dir = "/content/Blue Monday (Crossnaders Remix) - Extended Mix"  # Replace with your desired directory path

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to compute cosine similarity
def compute_similarity(audio_embeds, text_embeds):
    similarities = {
        label: torch.nn.functional.cosine_similarity(audio_embeds, text_embeds[label]).item()
        for label in text_embeds
    }
    return sorted(similarities.items(), key=lambda x: x[1], reverse=True)

genre_results = compute_similarity(audio_embeddings, genre_embeddings)

# Extract genre names
genre_names = [
    genre[0].split("genre,")[0].split("audio belongs to the")[1].strip()  # Extract genre name
    for genre in genre_results[:3]  # Top 3 genres
]

# Save to JSON file in the specified directory
output_file_path = os.path.join(output_dir, "Blue Monday (Crossnaders Remix) - Extended Mix_genre_predictions.json")
output = {"Genre Predictions": genre_names}
with open(output_file_path, "w") as json_file:
    json.dump(output, json_file, indent=4)

print(f"Genre predictions saved to {output_file_path}: {genre_names}")


Genre predictions saved to /content/Blue Monday (Crossnaders Remix) - Extended Mix/Blue Monday (Crossnaders Remix) - Extended Mix_genre_predictions.json: ['Hip Hop', 'Pop', 'Experimental']


In [ ]:
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the source directory (the entire directory you want to copy)
source_dir = "/content/Blue Monday (Crossnaders Remix) - Extended Mix"  # Replace with your source directory

# Define the Google Drive destination directory (including the folder name where you want the directory to go)
drive_folder_path = "/content/drive/My Drive/output/Blue Monday (Crossnaders Remix) - Extended Mix"  # Replace with your desired path in Google Drive

# Ensure the parent directory exists in Google Drive
os.makedirs(os.path.dirname(drive_folder_path), exist_ok=True)

# Use shutil.copytree to copy the entire directory and its contents
# Set dirs_exist_ok=True to overwrite any existing directory
shutil.copytree(source_dir, drive_folder_path, dirs_exist_ok=True)

print(f"Entire directory '{source_dir}' has been copied to '{drive_folder_path}'")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Entire directory '/content/Blue Monday (Crossnaders Remix) - Extended Mix' has been copied to '/content/drive/My Drive/output/Blue Monday (Crossnaders Remix) - Extended Mix'
